In [1]:
from src.dataset.dataset import SparkifyDataset

In [2]:
train_file = "data/train_dataset"

In [3]:
ds = SparkifyDataset(train_file)

In [4]:
from src.preprocess.collate_fn import mat_collate_fn

In [5]:
from torch.utils.data import DataLoader

In [6]:
dl = DataLoader(ds, batch_size=2, collate_fn=mat_collate_fn)

In [7]:
import json

In [8]:
unique_pages_file = "data/unique_pages.json"

In [9]:
with open(unique_pages_file, "r") as f:
    unique_pages = json.load(f)

In [10]:
len(unique_pages)

22

In [11]:
from src.nn.Transformer import TransformerModel

In [12]:
model = TransformerModel(
    num_classes=len(unique_pages),
    nhead=2,
    num_layers=2
)

In [13]:
from torch.optim.adam import Adam

In [14]:
opt = Adam(model.parameters(), lr=0.001)

In [15]:
from torch.nn.functional import cross_entropy

In [16]:
sample = next(iter(dl))

/run/media/john/Secondary/Documents/Cornell/Spring 2024/ORIE 4741 Learning From Data/Project/proj/lib64/python3.11/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  return _nested.nested_tensor(


In [17]:
emb, pos_indices, targets, mask = sample

In [18]:
output = model(emb, pos_indices, mask=mask)

In [19]:
output.shape

torch.Size([2, 128, 22])

In [20]:
loss = cross_entropy(output.reshape(-1, len(unique_pages)), targets.reshape(-1), ignore_index=-1)

In [21]:
loss

tensor(3.9768, grad_fn=<NllLossBackward0>)

In [22]:
opt.zero_grad()
loss.backward()
opt.step()

In [23]:
pred = output.argmax(dim=-1)

In [24]:
pred.shape

torch.Size([2, 128])

In [29]:
import torch

In [33]:
pred_mock = torch.tensor([
    [1, 0, 2, 3],
    [0, 2, 1, 3]
])

In [34]:
targets_mock = torch.tensor([
    [1, 2, 0, -1],
    [0, 2, 1, -1]
])

In [35]:
def accuracy(pred, targets):
    return (pred == targets).sum().item() / (targets != -1).sum().item()

In [38]:
(pred_mock==targets_mock).sum()

tensor(4)

In [40]:
(targets_mock != -1).sum()

tensor(6)